## Scenario 2: A cross-functional team with one data scientist working on an ML model
MLflow setup:

- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem
The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

mlflow server --backend-store-uri sqlite:///backend.db

In [2]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1720948382481, experiment_id='1', last_update_time=1720948382481, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1720948233631, experiment_id='0', last_update_time=1720948233631, lifecycle_stage='active', name='Default', tags={}>]

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/07/14 18:49:43 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'mlflow-artifacts:/2/a654b0826c0f455cb862fc355cb583f1/artifacts'


In [7]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1720982983219, experiment_id='2', last_update_time=1720982983219, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1720948382481, experiment_id='1', last_update_time=1720948382481, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1720948233631, experiment_id='0', last_update_time=1720948233631, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the model registry

In [8]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [9]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1720960650645, description='', last_updated_timestamp=1720961842077, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1720960650830, current_stage='Production', description='', last_updated_timestamp=1720961097169, name='nyc-taxi-experiment', run_id='837733a73ab040faaa4ecff356fc5283', run_link='', source='mlflow-artifacts:/1/837733a73ab040faaa4ecff356fc5283/artifacts/modles_mlflow', status='READY', status_message='', tags={}, user_id='', version='1'>,
  <ModelVersion: aliases=[], creation_timestamp=1720960659739, current_stage='Archived', description='', last_updated_timestamp=1720961842077, name='nyc-taxi-experiment', run_id='19dda245c7df4764aad5f2456fd89e9a', run_link='', source='mlflow-artifacts:/1/19dda245c7df4764aad5f2456fd89e9a/artifacts/modles_mlflow', status='READY', status_message='', tags={}, user_id='', version='2'>,
  <ModelVersion: aliases=[], creation_timestamp=1720960873010, current_stage='Staging', descript

In [12]:
run_id = "a654b0826c0f455cb862fc355cb583f1"
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/07/14 18:52:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1720983145691, current_stage='None', description='', last_updated_timestamp=1720983145691, name='iris-classifier', run_id='a654b0826c0f455cb862fc355cb583f1', run_link='', source='mlflow-artifacts:/2/a654b0826c0f455cb862fc355cb583f1/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>